## Example Training Notebook

##### Training

Only three things needed:
1. Dataset
2. Config Setup
3. Model


In [ ]:
%cd ../../
import os
import chessbot
from omegaconf import OmegaConf

from simple_chessnet import SimpleChessNet
from chessbot.train import ChessTrainer
from chessbot.common import DEFAULT_DATASET_DIR

cwd = os.getcwd()
dataset_path = os.path.join(cwd, 'dataset/ChessBot-Dataset-0.1.0/')
dataset_path = DEFAULT_DATASET_DIR # Can also use this if dataset is in default repo location

# Get the default OmegaConf config
cfg = chessbot.config.get_cfg()

# Alternatively, load a config from a file with omegaconf
cfg_load = OmegaConf.load('examples/example_model/config.yaml')

# Dataset
cfg.dataset.data_path = dataset_path
cfg.dataset.size_train = 1  # How many files to use for each training round. Adjust based on cpu mem
cfg.dataset.size_test = 1    # How many files to randomly sample for testing every validation_every iters
cfg.dataset.num_threads = 4   # Number of threads to use for data loading

# Train
cfg.train.rounds = 1
cfg.train.epochs = 1 
cfg.train.batch_size = 128 # Adjust based on GPU
cfg.train.lr = 0.001
cfg.train.compile = True
cfg.train.amp = 'bf16'

cfg.train.validation_every = 1
cfg.train.output_dir = 'output'

model = SimpleChessNet()
trainer = ChessTrainer(cfg, model)
trainer.train()

#### Evaluate Model

In [ ]:
import torch
from simple_chessnet import SimpleChessNet

from chessbot.models import align_state_dict
from chessbot.inference import evaluate_model

# Evaluate the model
batch_size = 3072
num_threads = 15
pgn_dir = DEFAULT_DATASET_DIR
weights = os.path.join(cfg.train.output_dir, "model_best", "pytorch_model.bin")

model = SimpleChessNet()
model.load_weights(weights)
model.eval()
model = torch.compile(model)

evaluate_model(
    model, 
    pgn_dir, 
    batch_size, 
    num_threads, 
    device='cuda')
